# Activity 

**Command line application to track students in a class: **Add, edit, view, remove

**Focus using:**
- enums, Options, Result, macth, interators, etc…
- Ownership, Borrowing issues
- Mutability

```
======manage student======
1. Add student
2. View students
3. Remove student
4. Update student

Enter selection:
```

```rust
use std::{collections::HashMap, io};

#[derive(Clone, Debug)]
pub struct Student {
    name: String,
    age: i32,
}

#[derive(Debug, Clone)]
pub struct Class {
    inner: HashMap<String, Student>,
}

impl Class {
    fn new() -> Self {
        Self {
            inner: HashMap::new(),
        }
    }

    fn add(&mut self, student: Student) {
        self.inner.insert(student.name.to_owned(), student);
    }

    fn get_all(&self) -> Vec<&Student> {
        self.inner.values().collect()
    }
    fn delete(&mut self, name: &str) -> bool {
        self.inner.remove(name).is_some()
    }
    fn update(&mut self, name: &str, age: i32) -> bool {
        match self.inner.get_mut(name) {
            Some(name) => {
                name.age = age;
                true
            }
            None => false,
        }
    }
}

mod manager {
    use crate::{get_input, get_input_int, Class, Student};

    pub fn add_student(class: &mut Class) {
        println!("please enter name student");
        let name = match get_input() {
            Some(name) => name,
            None => return,
        };
        let age = match get_input_int() {
            Some(age) => age,
            None => return,
        };
        let student = Student { name, age };
        class.add(student);
    }
    pub fn view_class(class: &Class) {
        for stu in class.get_all() {
            println!("{:?}", stu);
        }
    }

    pub fn del_student(class: &mut Class) {
        for stu in class.get_all() {
            println!("{:?}", stu);
        }
        println!("please enter name you want remove");
        let name = match get_input() {
            Some(input) => input,
            None => return,
        };
        if class.delete(&name) {
            println!("remove student");
        } else {
            println!("not found");
        }
    }

    pub fn edit_student(class: &mut Class) {
        for stu in class.get_all() {
            println!("{:?}", stu);
        }
        println!("please enter name you want update");
        let name = match get_input() {
            Some(input) => input,
            None => return,
        };

        println!("please enter age to edit");
        let age = match get_input_int() {
            Some(input) => input,
            None => return,
        };
        if class.update(&name, age) {
            println!("student has edit");
        } else {
            println!("not found");
        }
    }
}

fn get_input_int() -> Option<i32> {
    println!("enter age of student");
    let input = match get_input() {
        Some(input) => input,
        None => return None,
    };

    let parsed_input: Result<i32, _> = input.parse();
    match parsed_input {
        Ok(input) => Some(input),
        Err(_) => None,
    }
}

fn get_input() -> Option<String> {
    let mut buffer = String::new();
    while io::stdin().read_line(&mut buffer).is_err() {
        println!("Please enter your data again");
    }
    let input = buffer.trim().to_owned();
    if &input == "" {
        None
    } else {
        Some(input)
    }
}

enum MainMenu {
    AddStudent,
    ViewClass,
    RemoveStudent,
    UpdateStudent,
}

impl MainMenu {
    fn choice(input: &str) -> Option<MainMenu> {
        match input {
            "1" => Some(MainMenu::AddStudent),
            "2" => Some(MainMenu::ViewClass),
            "3" => Some(MainMenu::RemoveStudent),
            "4" => Some(MainMenu::UpdateStudent),
            _ => None,
        }
    }
    fn show_choice() {
        println!("");
        println!("== Class Manager ==");
        println!("1. Add Student");
        println!("2. View Class");
        println!("3. Remove Student");
        println!("4. Update Student");
        println!("");
        println!("Enter selection: ");
    }
}

fn main() {
    // let mut student = Class::new();
    let mut student = Class::new();
    loop {
        MainMenu::show_choice();
        let input = get_input().expect("No data");
        match MainMenu::choice(input.as_str()) {
            Some(MainMenu::AddStudent) => manager::add_student(&mut student),
            Some(MainMenu::ViewClass) => manager::view_class(&student),
            Some(MainMenu::RemoveStudent) => manager::del_student(&mut student),
            Some(MainMenu::UpdateStudent) => manager::edit_student(&mut student),
            None => return,
        }
    }
}
```

# Lifetimes 

**Lifetime**
- Lifetime là những gì compiler của rust sử dụng để theo dõi thời gian các tham chiếu có giá trị. Checking Reference là một trong những trách nhiệm chính của chức năng borrow checker’s. Lifetime giúp cho borrow checker’s đảm bảo giá trị mượn của không bao giờ có các reference ko hợp lệ.
- Tất cả dữ liệu trong rust đều có thời gian tồn tại nhưng trình biên dịch có thể tự động tính toán thời gian tồn tại trong nhiều trường hợp. Vì vậy bạn thường ko phải lúc nào cũng thấy nó trong code.

```rust
//syntax
const HANGSO: &'static i32 = 1124;
struct Name<'a> {
    field: &'a DataType,
}

fn add<'a,'b>(x: &'a i32, y: &'b i32) -> &'a i32 {
    x + y
}
```

- Convention use ‘a, ‘b, ‘c
- static is reserved - static cho biết dữ liệu tồn tại trong bộ nhớ toàn bộ thời gian của trường trình.
- &’a sẽ thông báo với trình biên dịch rằng dữ liệu này sẽ vẫn còn tồn tại sau khi structure này đã bị phá huỷ.

**Tại sao lại cần lifetimes**
- Rust’s onwership model
- Borrow checker’s sẽ đảm nhiệm việc cấp phát và giải phóng bộ nhớ và cũng đảm bảo rằng không có tham chiếu nào được trỏ tới bộ nhớ đã được giải phóng.
- Giống như borrow data, chúng được kiểm tra tại thời gian biên dịch, nên nếu không hợp lệ sẽ không chạy được chường trình.
- Lifetime cực kì quan trọng khi function trả về một tham chiếu, và khi Struct chưa một dữ liệu tham chiếu.

**Lifetimes Struct & impl**
Demo 

```rust
#[derive(Debug)]
struct Cards {
    inner: Vec<IdCard>,
}

#[derive(Debug, Eq, PartialEq, PartialOrd, Ord)]
enum City {
    HCM,
    HN,
    DN,
}

#[derive(Debug)]
struct IdCard {
    name: String,
    age: u8,
    city: City,
}

impl IdCard {
    pub fn new(name: &str, age: u8, city: City) -> Self {
        Self {
            name: name.to_string(),
            age,
            city,
        }
    }
}

fn new_ids() -> Cards {
    Cards {
        inner: vec![
            IdCard::new("Dang", 27, City::HCM),
            IdCard::new("Quang", 28, City::DN),
            IdCard::new("Vu", 29, City::HN),
            IdCard::new("Ok", 30, City::HN),
            IdCard::new("Khong", 31, City::HCM),
        ],
    }
}

fn main() {
    let ids = new_ids();
}
```

Active 

```rust
#[derive(Debug)]
struct Cards {
    inner: Vec<IdCard>,
}

#[derive(Debug, Eq, PartialEq, PartialOrd, Ord)]
enum City {
    HCM,
    HN,
    DN,
}

#[derive(Debug)]
struct IdCard {
    name: String,
    age: u8,
    city: City,
}

impl IdCard {
    pub fn new(name: &str, age: u8, city: City) -> Self {
        Self {
            name: name.to_string(),
            age,
            city,
        }
    }
}

fn new_ids() -> Cards {
    Cards {
        inner: vec![
            IdCard::new("Dang", 27, City::HCM),
            IdCard::new("Quang", 28, City::DN),
            IdCard::new("Vu", 29, City::HN),
            IdCard::new("Ok", 30, City::HN),
            IdCard::new("Khong", 31, City::HCM),
        ],
    }
}

#[derive(Debug)]
struct YoungPeople<'a> {
    inner: Vec<&'a IdCard>,
}

#[derive(Debug)]
struct OldPeople<'a> {
    inner: Vec<&'a IdCard>,
}

impl<'a> YoungPeople<'a> {
    fn living_in_hcm(&self) -> Self {
        Self {
            inner: self
                .inner
                .iter()
                .filter(|id| id.city == City::HCM)
                .map(|id| *id)
                .collect(),
        }
    }
}

fn main() {
    let ids = new_ids();
    let young = YoungPeople {
        inner: ids.inner.iter().filter(|id| id.age <= 28).collect(),
    };
    for id in ids.inner.iter() {
        println!("{:?}", id);
    }

    println!("\nyoung people\n");
    for id in young.inner.iter() {
        println!("{:?}", id);
    }

    println!("\n living in hcm");
    for id in young.living_in_hcm().inner.iter() {
        println!("{:?}", id);
    }
}
```

# Improving Program Reliability

## Manual error creation

```rust
#[derive(Debug)]
enum LockError {
    MechainError(i32),
    NetworkError,
    NotAuthorized,
}

use std::fmt;
impl fmt::Display for LockError {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            Self::MechainError(code) => write!(f, "mechaine error: {}", code),
            Self::NetworkError => write!(f, "network error"),
            Self::NotAuthorized => write!(f, "authorized error"),
        }
    }
}

fn main() {}
```

## use ‘thiserror’ crate

```rust 
thiserror = "*"
```

```rust 
use thiserror::Error;

#[derive(Debug, Error)]
enum LockError {
    #[error("Machine error: {0}")]
    MachineError(i32),
    #[error("Network error")]
    Network(#[from] NetworkError),
    #[error("Authorized error")]
    NotAuthorized,
}

#[derive(Debug, Error)]
enum NetworkError {
    #[error("Connecting time out")]
    TimeOut,
    #[error("Unreachable")]
    Unreachable,
}

fn maybe_some(a: Option<&str>) -> Result<String, LockError> {
    if a.is_some() {
        Ok("Is Some".to_owned())
    } else {
        Err(LockError::Network(NetworkError::TimeOut))
    }
}

fn main() {
    let some = None;
    match maybe_some(some) {
        Ok(data) => println!("{}", data),
        Err(e) => println!("{}", e),
    }
}
```

# Array & Slices

## Arrays 

- Arrays Đại diện cho một vùng bộ nhớ liền kề nhau nằm trên heap.
- Tất cả phần từ của một array phải có cùng kích thước. nghĩa là các phần tử đó phải chung 1 kiểu dữ liệu.
- Các array không có dữ liệu dynamic.
- Các dữ liệu của array được mã hoá cứng trong program trong hầu hết các trường hợp.
- Thông thường thì bạn sẽ muốn sử dụng vector hơn là array.
- Tuy nhiên arrays rất hữu ích trong việc sử dụng trong networks protocol, crypto algorithm, và matrices.


```rust
fn main() {
    let months = ["January", "February", "March", "April", "May", "June", "July",
                  "August", "September", "October", "November", "December"];
    let a: [i32; 5] = [1, 2, 3, 4, 5];
}
```

## Slices 

- là dạng chế độ xem mượn dữ liệu từ arrays or String.
- Slices có thể là tạo ra iterator.
- Indices giới hạn bởi arrays.

```rust
fn main() {
    let months = ["January", "February", "March", "April", "May", "June", "July",
                  "August", "September", "October", "November", "December"];
    let a: [i32; 5] = [1, 2, 3, 4, 5];
    let slice = &[a]
}
```

**Activity**

```rust 
trait Perimeter {
    fn calculate_perimeter(&self) -> i32;
}

struct Square {
    side: i32,
}

impl Square {
    fn new(side: i32) -> Self {
        Self { side }
    }
}

impl Default for Square {
    fn default() -> Self {
        Self { side: 40 }
    }
}

impl Perimeter for Square {
    fn calculate_perimeter(&self) -> i32 {
        self.side * 4
    }
}

struct Triangle {
    side_a: i32,
    side_b: i32,
    side_c: i32,
}
impl Perimeter for Triangle {
    fn calculate_perimeter(&self) -> i32 {
        self.side_a + self.side_b + self.side_c
    }
}

fn print_perimeter(shape: impl Perimeter) {
    let perimeter = shape.calculate_perimeter();
    println!("perimeter : {:?}", perimeter);
}

fn main() {
    let square = Square::default();
    let triangle = Triangle {
        side_a: 3,
        side_b: 4,
        side_c: 5,
    };
    print_perimeter(square);
    print_perimeter(triangle);
}
```